# SOEN_PROJECT

In [1]:
# Importing libraries and setting the path.

import os
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from sklearn.metrics import confusion_matrix


import warnings

warnings.filterwarnings('ignore')

path = 'archive/A_DeviceMotion_data/A_DeviceMotion_data/'

/Users/shubhangkhattar/opt/anaconda3/lib/python3.9/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Reading of files 

In [2]:
# Setting up lists for importing the data.

list_of_folders = os.listdir(path)
list_of_directories = [path + i for i in list_of_folders]
list_of_pdfs = []


activities = ['dws', 'ups', 'sit', 'std', 'wlk', 'jog']

# spark = SparkSession.builder.appName('SOEN_PROJECT_').getOrCreate()

spark = (
    SparkSession.builder
        .appName('SOEN_PROJECT_2')
        .config('spark.driver.memory', '5g')
        .config('spark.executor.memory', '5g')
        .config('spark.dynamicAllocation.enabled', True)
        .config('spark.dynamicAllocation.maxExecutors', 25)
        .config('spark.executor.memoryOverhead', '4096')
        .getOrCreate()
)
spark = spark.sparkContext

for i in list_of_directories:
           
    if ".DS_Store" in i:
        continue
    
    list_of_csv = os.listdir(i)
    
    for j in list_of_csv:
        
        k = ps.read_csv(i + '/' + j, sep=',', header="infer").rename(columns={"_c0":"time"})
        
        k['subject'] = int(j.split('_')[1].split('.')[0])
        k['activity'] = i.split('/')[-1].split('_')[0]
        k['trial'] = int(i.split('/')[-1].split('_')[1])
        k.insert(0, 'accel_x', k['userAcceleration.x'] + k['gravity.x'])
        k.insert(1, 'accel_y', k['userAcceleration.y'] + k['gravity.y'])
        k.insert(2, 'accel_z', k['userAcceleration.z'] + k['gravity.z'])
        k.insert(3, 'accel_norm', np.sqrt(k['accel_x']**2 + k['accel_y']**2 + k['accel_z']**2))
        list_of_pdfs.append(k)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/12 19:18:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pdf = ps.concat(list_of_pdfs).set_index('time')
pdf = pdf.drop(['attitude.roll', 'attitude.pitch', 'attitude.yaw', 'gravity.x', 'gravity.y', 'gravity.z', 'userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z'], axis=1)
pdf = pdf.rename(columns={'rotationRate.x': 'gyro_x', 'rotationRate.y': 'gyro_y', 'rotationRate.z': 'gyro_z'})

In [4]:
pdf.head(20)

23/03/12 19:24:37 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


23/03/12 19:24:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv


,accel_x,accel_y,accel_z,accel_norm,gyro_x,gyro_y,gyro_z,subject,activity,trial
time,,,,,,,,,,
0,-0.122085,0.626434,-0.161957,0.658448,-1.264215,-1.027909,-0.947909,12,dws,11
1,-0.160369,0.946457,-0.288605,1.002393,-1.162024,-0.269118,-0.848823,12,dws,11
2,-0.178482,1.202301,-0.178314,1.228487,-0.665042,0.520170,-0.726722,12,dws,11
3,-0.033493,1.307663,-0.223786,1.327096,-0.079809,0.055322,-0.604534,12,dws,11
4,-0.141067,1.322677,-0.641449,1.476764,0.456097,-0.186877,-0.441315,12,dws,11
5,-0.020950,1.711899,-0.444931,1.768898,1.531273,-0.369080,-0.563895,12,dws,11
6,0.137573,1.897262,-0.248901,1.918458,2.413919,-0.488344,-0.275566,12,dws,11
7,0.136551,1.647385,-0.086471,1.655295,3.309900,-0.802186,0.175475,12,dws,11
8,-0.477051,1.472824,-0.042968,1.548753,3.873452,-0.347702,0.582617,12,dws,11


In [5]:
df_spark = pdf.to_spark(index_col="time")
df_spark.show(5)

23/03/12 19:25:18 WARN DAGScheduler: Broadcasting large task binary with size 4.8 MiB
23/03/12 19:25:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
+----+--------------------+------------------+--------------------+------------------+-------------------+--------------------+--------------------+-------+--------+-----+
|time|             accel_x|           accel_y|             accel_z|        accel_norm|             gyro_x|              gyro_y|              gyro_z|subject|activity|trial|
+----+------------------

In [6]:
assembler = VectorAssembler(
    inputCols=["time","accel_x","accel_y","accel_z","accel_norm","gyro_x","gyro_y","gyro_z","subject","trial"],
    outputCol="features")

In [7]:
df_spark = assembler.transform(df_spark)

In [8]:
df_spark.show(5)

23/03/12 19:25:34 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/03/12 19:25:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
+----+--------------------+------------------+--------------------+------------------+-------------------+--------------------+--------------------+-------+--------+-----+--------------------+
|time|             accel_x|           accel_y|             accel_z|        accel_norm|             gyro_x|              gyro_y|              gyro_z|subject|activity|trial|    

In [9]:
string_indexer = StringIndexer(inputCol='activity', outputCol='label')
StringIndexerModel = string_indexer.fit(df_spark)
df_spark = StringIndexerModel.transform(df_spark)

23/03/12 19:25:45 WARN DAGScheduler: Broadcasting large task binary with size 1430.0 KiB


In [10]:
df_spark.show(2)

23/03/12 19:26:15 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/03/12 19:26:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
+----+--------------------+------------------+--------------------+------------------+-------------------+--------------------+---------+-------+--------+-----+--------------------+-----+
|time|             accel_x|           accel_y|             accel_z|        accel_norm|             gyro_x|              gyro_y|   gyro_z|subject|activity|trial|            features

# Split the Data

In [11]:
train_df = df_spark.where(df_spark.trial <= 10)
test_df = df_spark.where(df_spark.trial > 10)

In [12]:
train_df = train_df.select("features","label")
test_df = test_df.select("features","label")

In [13]:
train_df.show(2)

23/03/12 19:26:40 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/03/12 19:26:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
23/03/12 19:26:40 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/03/12 19:26:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, 

23/03/12 19:26:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/wlk_7/sub_3.csv
23/03/12 19:26:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/D

23/03/12 19:26:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/wlk_7/sub_21.csv
23/03/12 19:26:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/std_14/sub_20.csv
23/03/12 19:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

23/03/12 19:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/wlk_15/sub_11.csv
23/03/12 19:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

23/03/12 19:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/wlk_15/sub_5.csv
23/03/12 19:26:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:26:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/wlk_15/sub_23.csv
23/03/12 19:26:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

In [14]:
test_df.show(2)

23/03/12 19:27:03 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
23/03/12 19:27:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,-0.122084999...|  5.0|
|[1.0,-0.160369000...|  5.0|
+--------------------+-----+
only showing top 2 rows



In [15]:
df_spark.count()

1412865

In [16]:
train_df.count()

1081446

In [17]:
test_df.count()

331419

In [24]:
train_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)

23/03/12 20:21:12 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 188405 ms exceeds timeout 120000 ms
23/03/12 20:21:12 WARN SparkContext: Killing executors is not supported by current scheduler.
23/03/12 20:21:16 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:643)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1057)
	at 

# Decison Tree

In [18]:
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="label")
dtc = dtc.fit(train_df)

23/03/12 19:28:09 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/12 19:28:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
23/03/12 19:28:10 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/12 19:28:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_13.csv
23/03/12 19:28:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

23/03/12 19:28:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_7.csv
23/03/12 19:28:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:28:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_18.csv
23/03/12 19:28:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

...

23/03/12 19:30:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/std_6/sub_15.csv
23/03/12 19:30:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:31:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/std_6/sub_19.csv
23/03/12 19:31:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:31:02 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/12 19:31:34 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/12 19:32:09 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


### Evaluating perfromance

In [19]:
pred = dtc.transform(test_df)

In [20]:
pred.show(3)

23/03/12 19:32:57 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB


23/03/12 19:32:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_12.csv
+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0,-0.122084999...|  5.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       4.0|
|[1.0,-0.160369000...|  5.0|[0.0,0.0,0.0,0.0,...|[0.0,0.0,0.0,0.0,...|       4.0|
|[2.0,-0.178481999...|  5.0|[0.0,0.0,0.0,0.0

In [21]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
acc = evaluator.evaluate(pred)
print("Prediction Accuracy: ", acc)

y_pred=pred.select("prediction").collect()
y_orig=pred.select("label").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

23/03/12 19:33:15 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/12 19:33:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_11.csv
23/03/12 19:33:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

23/03/12 19:33:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_7.csv
23/03/12 19:33:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:33:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_15.csv
23/03/12 19:33:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

23/03/12 19:33:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/dws_11/sub_18.csv
23/03/12 19:33:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar

...

23/03/12 19:34:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/std_6/sub_4.csv
23/03/12 19:34:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/D

23/03/12 19:34:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/std_6/sub_21.csv
23/03/12 19:34:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:34:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/Desktop/SOEN6111_PROJECT/archive/A_DeviceMotion_data/A_DeviceMotion_data/std_6/sub_22.csv
23/03/12 19:34:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
 Schema: _c0, gravity.x, gravity.y, gravity.z, rotationRate.x, rotationRate.y, rotationRate.z, userAcceleration.x, userAcceleration.y, userAcceleration.z
Expected: _c0 but found: 
CSV file: file:///Users/shubhangkhattar/

23/03/12 19:35:04 WARN DAGScheduler: Broadcasting large task binary with size 1161.4 KiB


Confusion Matrix:
[[     0      0      0      0  65272      0]
 [     0      0      0      0 104121      0]
 [     0      0      0      0  81611      0]
 [     0      0      0      0  28009      0]
 [     0      0      0      0  29904      0]
 [     0      0      0      0  22502      0]]
